# Работа со строковыми значениями



Материалы:

* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Разминка

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
# Находим максимальную длину ключа для выравнивания
max_key_length = max(len(key) for key in obj)

# Выводим данные в нужном формате
for key, value in obj.items():
    print(f'{key:<{max_key_length}} = "{value}"')

home_page = "https://github.com/pypa/sampleproject"
keywords  = "sample setuptools development"
license   = "MIT"


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [2]:
# obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
# obj

import pandas as pd
import re

# Создаем Series
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])

# Функция для поиска номера группы
def find_group_number(text):
    # Обновленное регулярное выражение
    match = re.search(r'\b[Пп][Мм]?d{2}-d\b|\bd{2}-d\b', text)
    return match.group(0) if match else None

# Применяем функцию к Series
group_numbers = obj.apply(find_group_number)

print(group_numbers)




0    None
1    None
2    None
dtype: object


C:\Users\berni\AppData\Local\Temp\ipykernel_72188\516788666.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [3]:
import pandas as pd

# Загрузка данных из файла
recipes = pd.read_csv('recipes_sample.csv')

# Выбор 5 случайных рецептов
random_recipes = recipes.sample(n=5)

# Извлечение необходимых столбцов
result = random_recipes[['id', 'minutes']]

# Определение ширины столбцов
id_width = max(result['id'].astype(str).map(len).max(), len('id')) + 2
minutes_width = max(result['minutes'].astype(str).map(len).max(), len('minutes')) + 2

# Форматированный вывод
header = f"| {'id'.center(id_width)} | {'minutes'.center(minutes_width)} |"
separator = f"|{'-' * (id_width - 2)}|{'-' * (minutes_width - 2)}|"
rows = [f"| {str(row['id']).center(id_width)} | {str(row['minutes']).center(minutes_width)} |" for index, row in result.iterrows()]

# Печать таблицы
print(header)
print(separator)
for row in rows:
    print(row)


|    id    |  minutes  |
|------|-------|
|  191561  |     60    |
|  304674  |     40    |
|  25516   |     50    |
|  47422   |     35    |
|  86867   |     5     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

In [4]:
import csv
import xml.etree.ElementTree as ET

def read_recipes_from_csv(filename):
    recipes = {}
    with open(filename, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            recipe_id = int(row['id'])
            recipes[recipe_id] = {
                'name': row['name'],
                'contributor_id': row['contributor_id'],
                'minutes': int(row['minutes'])
            }
    return recipes

def read_steps_from_xml(filename):
    steps = {}
    tree = ET.parse(filename)
    root = tree.getroot()
    
    for recipe in root.findall('recipe'):
        # Извлекаем id из элемента <id>
        recipe_id_str = recipe.find('id').text
        if recipe_id_str is None:
            continue  # Пропускаем, если id отсутствует

        recipe_id = int(recipe_id_str)
        # Извлекаем шаги из элемента <steps>
        recipe_steps = [step.text for step in recipe.find('steps').findall('step')]
        steps[recipe_id] = recipe_steps
    
    return steps

def show_info(recipe_id, recipes, steps):
    # Получаем рецепт и шаги по ID
    recipe = recipes.get(recipe_id)
    recipe_steps = steps.get(recipe_id)

    if not recipe or recipe_id not in steps:
        return f"Рецепт с ID {recipe_id} не найден."

    # Форматируем название
    formatted_name = ' '.join(word.capitalize() for word in recipe["name"].split())

    # Формируем строку с шагами
    steps_str = '\n'.join(f"{i + 1}. {step.capitalize()}" for i, step in enumerate(recipe_steps))

    # Формируем финальную строку
    result = (
        f'"{formatted_name}"\n\n'
        f'{steps_str}\n'
        '----------\n'
        f'Автор: {recipe["contributor_id"]}\n'
        f'Среднее время приготовления: {recipe["minutes"]} минут\n'
    )

    return result

# Чтение данных из файлов
recipes = read_recipes_from_csv('recipes_sample.csv')
steps = read_steps_from_xml('steps_sample.xml')

# Вызов функции для рецепта с id 170895 (пример ID)
output = show_info(170895, recipes, steps)
print(output)


"Leeks And Parsnips Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addit

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [7]:
import pandas as pd
import re

# Загрузка данных из CSV файла
recipes = pd.read_csv('recipes_sample.csv')

# Фильтрация рецепта по ID
recipe_id = 67664
recipe_steps = recipes.loc[recipes['id'] == recipe_id, 'n_steps']

# Проверка, есть ли шаги для данного ID
if recipe_steps.empty:
    print(f"Нет шагов для рецепта с ID {recipe_id}.")
else:
    # Регулярное выражение для поиска
    pattern = r'(d+)s+(hour|hours|minute|minutes)'

    # Список для хранения результатов
    results = []

    # Поиск по каждому шагу рецепта
    for step in recipe_steps:
        # Проверяем, является ли шаг строкой
        if isinstance(step, str):
            matches = re.findall(pattern, step)
            results.extend(matches)  # Добавляем найденные результаты в список
        else:
            print(f"Пропущено значение: {step} (не строка)")

    # Вывод результатов
    if results:
        for result in results:
            print(f"{result[0]} {result[1]}")
    else:
        print("Совпадений не найдено.")


Пропущено значение: nan (не строка)
Совпадений не найдено.


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [8]:
import pandas as pd

# Загрузка данных из CSV файла
recipes = pd.read_csv('recipes_sample.csv')

# Регулярное выражение для поиска шаблона
pattern = r'^this[ws]*, ?but'

# Фильтрация рецептов, соответствующих шаблону
matching_recipes = recipes[recipes['description'].str.contains(pattern, regex=True, na=False)]

# Вывод количества найденных рецептов и примеров
count = matching_recipes.shape[0]
print(f"Количество подходящих рецептов: {count}")

# Выводим 3 примера подходящих описаний
examples = matching_recipes['description'].head(3).tolist()
for example in examples:
    print(example)


Количество подходящих рецептов: 0


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [10]:
import pandas as pd
import re

# Загрузка данных из CSV файла
recipes = pd.read_csv('recipes_sample.csv')

# Получаем шаги рецепта с id 72367 и 44123
recipe_id = 44123
recipe_steps = recipes.loc[recipes['id'] == recipe_id, 'n_steps'].values

if len(recipe_steps) > 0:
    # Предполагаем, что 'n_steps' содержит текст шагов рецепта
    steps_text = recipe_steps[0]

    # Проверка типа данных (декодирование не требуется)
    if isinstance(steps_text, bytes):
        steps_text = steps_text.decode('utf-8')

    # Теперь можно безопасно использовать регулярное выражение для замены пробелов
    modified_steps = re.sub(r's+', '/', steps_text)

    # Вывод измененных шагов
    print("Измененные шаги рецепта с id 44123:")
    print(modified_steps)
else:
    print(f"Рецепт с id {recipe_id} не найден.")




Измененные шаги рецепта с id 44123:
12 / 9


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [15]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Скачиваем необходимые ресурсы NLTK
nltk.download('punkt')

# Загрузка данных из CSV файла
recipes = pd.read_csv('recipes_sample.csv')

# Создаем множество для хранения уникальных слов
unique_words = set()

# Проходим по всем шагам рецептов
for steps in recipes['n_steps']:  # Убедитесь, что 'n_steps' - это правильное имя столбца
    # Проверяем, что значение не является NaN
    if isinstance(steps, str):  # Убедитесь, что это строка
        # Токенизируем текст на слова
        words = word_tokenize(steps)

        # Добавляем в множество только те слова, которые являются алфавитными и в нижнем регистре
        for word in words:
            if word.isalpha():  # Проверяем, является ли слово алфавитным
                unique_words.add(word.lower())  # Приводим к нижнему регистру

# Выводим количество уникальных слов
print(f"Количество уникальных слов среди всех рецептов: {len(unique_words)}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\berni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\berni/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\berni\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [13]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

# Загрузка данных из CSV файла
recipes = pd.read_csv('recipes_sample.csv')

# Создаем список для хранения количества предложений для каждого рецепта
sentence_counts = []

# Проходим по всем описаниям рецептов
for description in recipes['description']:  # Предполагается, что колонка с описаниями называется 'description'
    # Токенизируем текст на предложения
    sentences = sent_tokenize(description)
    # Сохраняем количество предложений и сам рецепт
    sentence_counts.append((len(sentences), description))

# Преобразуем в DataFrame для удобства сортировки
sentence_counts_df = pd.DataFrame(sentence_counts, columns=['sentence_count', 'description'])

# Находим 5 рецептов с наибольшим количеством предложений
top_5_recipes = sentence_counts_df.nlargest(5, 'sentence_count')

# Выводим результаты
print(top_5_recipes)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\berni/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\berni\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [14]:
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')

def print_pos_tags(sentence):
    # Токенизируем предложение на слова
    words = nltk.word_tokenize(sentence)
    # Получаем части речи для токенов
    pos_tags = nltk.pos_tag(words)
    
    # Извлекаем только части речи
    pos_list = [tag for word, tag in pos_tags]
    
    # Форматируем вывод
    pos_line = '   '.join(pos_list)
    words_line = ' '.join(words)
    
    # Выводим результат
    print(pos_line)
    print(words_line)

# Загрузка данных из CSV файла
recipes = pd.read_csv('recipes_sample.csv')

# Получаем название рецепта с id 241106
recipe_title = recipes.loc[recipes['id'] == 241106, 'name'].values[0]

# Проверяем работоспособность функции на названии рецепта
print_pos_tags(recipe_title)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\berni\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\berni/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\berni\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
